# Classroom problem

In [ ]:
#using Pkg
#Pkg.add("StatsBase");

## General code for solving Itô integrals with the Euler-Maruyama method (no boundary conditions (BCs) included)

In [ ]:
de_EM_solver = function(f, g, x₀, p, T, Δt)

    """
    f = f(x, p, t) and g = g(x, p, t), where x is the process variable, p stands for a vector of parameters and t is time.    
    Outputs:
        tt - a partitioned time interval [0, T]
        xx - a solution to the stochastic integral dX_t = f(X_t, p, t)dt + g(X_t, p, t)dW
    """
    
    x = copy(x₀)
    t = 0.0

    Nx = length(x) 
    tt = 0.0:Δt:T
    xx = zeros(length(tt), Nx)

    xx[1, :] .= x
    
    Nw = length(p)-1
    
    for n in 1:length(tt)-1
        t⁻ = tt[n]
        x⁻ = xx[n, :]
        xx[n+1, :] = x⁻ + f(x⁻, p, t⁻).*Δt + sqrt(Δt)*g(x⁻, p, t⁻)*randn(Nw, 1)
    end
    
    return tt, xx
end

## 1. System with multiple Favourable States (Homework from week 3 revisited)

To simplify the problem, let us work with the interval $t\in[0,5]$.

$\frac{dx}{dt} = -\frac{κ₁}{6V^2}*x^3+\frac{\kappa_2}{2V}*x^2-κ₃*x+κ₄V,\quad x_0 = 0$

In [ ]:
SM = [-1 1 -1 1]

In [ ]:
Δt = 0.01
κ₁ = 0.0015
κ₂ = 0.36
κ₃ = 37.5
κ₄ = 2200
V = 0.5

p = [κ₁,κ₂,κ₃,κ₄, V]
x₀ = zeros(1)
T = 5.0


fode = function(x, p, t) 
    κ₁,κ₂,κ₃,κ₄,V = p
    return ####                                         # drift term
end
gode = function(x, p, t)
    Nx = length(x)
    Nw = length(p)-1
    return (zeros(Nx, Nw))
end
tt, xx_ode = de_EM_solver(fode, gode, x₀, p, T, Δt);

In [ ]:
using Plots
plot(tt, xx_ode, colour=:black, ylabel="X(t)", xlabel="t", label = "ODE")

In [ ]:
using LinearAlgebra

fsde = function(x, p, t) 
    κ₁,κ₂,κ₃,κ₄,V = p
    return ####                                         # drift term (same as for ODE)
end
gsde = function(x, p, t)
    κ₁,κ₂,κ₃,κ₄,V = p
    Nw = length(p)-1
    return ####                                         # diffusion matrix (1x4)
end
tt, xx_sde = de_EM_solver(fode, gsde, x₀, p, T, Δt);

In [ ]:
plot(tt, xx_sde, label="SDE", xlabel="t", ylabel="X(t)")
plot!(tt, xx_ode, colour=:black, linewidth=5, label="ODE")

Now, change the volume $V$ to $20V$. What is the effect of chaning the volume $V$?

In [ ]:
V = ####
p = [κ₁,κ₂,κ₃,κ₄, V]

tt, xx_ode = de_EM_solver(fode, gode, x₀, p, T, Δt)
tt, xx_sde = de_EM_solver(fsde, gsde, x₀, p, T, Δt);

In [ ]:
plot(tt, xx_sde, label="SDE", xlabel="t", ylabel="X(t)")
plot!(tt, xx_ode, colour=:black, linewidth=5, label="ODE")

## Approximation of the solution to the chemical master equation with multiple realisations of the solution to the SDE.

Let the vessel have its original volume $V=0.5$. 

In [ ]:
V = ####
p = [κ₁,κ₂,κ₃,κ₄, V]

## General code for solving Itô integrals with the Euler-Maruyama method (BCs included)

In [ ]:
de_EM_solver_bc = function(f, g, x₀, p, T, Δt)

    """
    f = f(x, p, t) and g = g(x, p, t), where x is the process variable, p stands for a vector of parameters and t is time.    
    Outputs:
        tt - a partitioned time interval [0, T]
        xx - a solution to the stochastic integral dX_t = f(X_t, p, t)dt + g(X_t, p, t)dW
    """
    
    tt = 0.0:Δt:T
    Nt = length(tt)

    x = copy(x₀)
    Nx = length(x) 
    xx = zeros(length(tt), Nx)

    xx[1, :] .= x
    
    Nw = length(p)-1
    
    for n in 1:length(tt)-1
        t⁻ = tt[n]
        x⁻ = xx[n, :]
        
        ####
        #### check whether the particle has escaped from the positive quadrant
        ####
        
    end
    
    return tt, xx
end

In [ ]:
Nr = 5000

tt = 0.0:Δt:T
Nt = length(tt)

Nx = length(x₀)

using SharedArrays
count_sde = SharedArray{Float64}(Nr, Nt, Nx);

In [ ]:
for nr in 1:Nr
    tt, xx_sde = de_EM_solver_bc(fode, gsde, x₀, p, T, Δt);
    count_sde[nr, :] = xx_sde
end

In [ ]:
count_sde[1, :]'

### Place the particle positions in histogram bins

In [ ]:
Nb = ####                                           # number of bins
upperbound = ###                                   # upper limit for the copy number of A
xx = range(.0, stop = upperbound, length = Nb+1)   # bin limits
Δx = xx[2]-xx[1]
xx_centre = xx[1:end-1] .+ Δx/2                    # bin centroids

In [ ]:
particles_counter = function(x::AbstractVector, xedges::AbstractRange)
    # Calculate bin index from x value
    N = size(x, 1)
    nxbins = length(xedges)-1
    xmin, xmax = extrema(xedges)
    δ𝑖δx = nxbins / (xmax - xmin)
    
    particle_density = zeros(nxbins)
   
    # Calculate the means for each bin, ignoring NaNs
    @inbounds for n ∈ eachindex(x)
        𝑖 = (x[n] - xmin) * δ𝑖δx
        if (0 <= 𝑖 < nxbins)  
           i = ceil(Int, 𝑖)
           particle_density[i] += ####
        end
    end
  
    Δx = xedges[2]-xedges[1]
    return ####
end

#### For the full evolution

In [ ]:
pp_sde = zeros(Nt, Nb) # number or time intervals and number of centroids

for t_id in 1:Nt
    pp_sde[t_id, :] = particles_counter(count_sde[:, t_id], xx)
end

In [ ]:
plot(xx_centre, pp_sde[2, :]./maximum(pp_sde[2, :]))
plot!(xx_centre, pp_sde[20, :]./maximum(pp_sde[20, :]))
plot!(xx_centre, pp_sde[501, :]./maximum(pp_sde[501, :]))

In [ ]:
using Plots
anim = @animate for k in 1:length(tt)
    plot(xx_centre, pp_sde[k, :], linetype=:steppost,
            title="t: ".*"$(round(tt[k], digits = 1))".*" minutes",
            label = "Empirical distribution with SDEs",
            colour = :black,
            xlabel = "nA",
            ylabel = "p(nA)") # 2:end since end = 1, periodic condition
end
gif(anim)

## Fokker-Planck for the same scenario


In [ ]:
#import Pkg; Pkg.add(["DiffEqOperators", "DifferentialEquations"])

In [3]:
x₀ = zeros(1)

using Distributions
init_dist = Uniform(x₀[]-1.0, x₀[]+1.0)
#init_dist = Dirac(x₀[])
prior_pdf = x -> pdf(init_dist, x)
pp₀ = prior_pdf.(xx_centre);
pp₀ ./= sum(pp₀*Δx)


ffunc, gfunc = fsde, gsde

M = size(Dfunc(zeros(length(x₀)), p, 0.), 2)
ff = Array{Float64}(undef, Nb)
gg = Array{Float64}(undef, Nb, M, M)

for n in 1:Nb   
    ff[n] = ffunc(xx_centre[n], p, 0.) 
    auxtensor = gfunc(xx_centre[n], p, 0.)    
    auxtensor = diagm(auxtensor[1,:])
    gg[n, :, :] = auxtensor*auxtensor'
end

LoadError: UndefVarError: xx_centre not defined

In [4]:
using DiffEqOperators

# Second order approximation to the second derivative
order = 2

deriv = 1
#deriv1 = UpwindDifference(deriv, order, Δx, Nx+1)
deriv1 = CenteredDifference(deriv, order, Δx, Nb)

deriv = 2
deriv2 = CenteredDifference(deriv, order, Δx, Nb)

#bc = DirichletBC(0.,0.)
#nc = NeumannBC(0.,0.)

LoadError: UndefVarError: Δx not defined

In [1]:
FP_assemble = function(pp, μ, D)
    pp ./= sum(pp*Δx)
    
    # Drift terms    
    μμ = broadcast(*, μ,pp)
    #μμ = bc*μμ
    μμ = [0; μμ; 0]
    
    μμ = deriv1*μμ

    # Diffusion terms
    DD = zeros(size(μμ))
    for m in 1:M
        DDm = broadcast(*,0.5*D[:, m, m],pp)
        # DDm = bc*DDm
        DDm = [0; DDm;0]
        DDm = deriv2*DDm
        DD += DDm
    end
    
    return -μμ+DD
end

## Construct the RHS of the PDE
function fpsystem!(du,u,p,t)
    du .= FP_assemble(u, ff, gg)
end

fpsystem! (generic function with 1 method)

In [ ]:
using DifferentialEquations

sol = solve(ODEProblem(fpsystem!, pp₀ , (0, T)), Tsit5(), saveat = tt)

In [ ]:
using Plots
anim = @animate for t_idx in 1:length(tt)
    plot(xx_centre, pp_sde[t_idx, :], linetype=:steppre,
            title="t: ".*"$(round(tt[t_idx], digits = 1))".*" minutes",
            label = "Empirical distribution with SDEs",
            colour = :black,
            xlabel = "nA",
            ylabel = "p(nA)") # 2:end since end = 1, periodic condition
    plot!(xx_centre, sol(tt[t_idx]), label = "Solution to the FP equation", colour = :darkorange)
end
gif(anim)

# Homework

In [ ]:
using Distributions

function velocityjump_2d(x₀, θ₀, dτ, κ, maxnumberofjumps)

  x = copy(x₀)
  θ = copy(θ₀)

  θ_dist = VonMises(####, ####)

  t = 0.0
  tt = [t]
  
  xx = copy(x)     
  
  k=1
  while (k <= maxnumberofjumps)
      
      τ = ####                                   
      t += τ
    
      θ += rand(θ_dist) 
      θ_dist = VonMises(####, ####)
        
      x = x .+ ####
    
      append!(tt, t)
      xx = cat(xx, x, dims=2)
      k += 1
  end
  
  return tt, xx'
end

In [ ]:
λ = 1
s = 1

# 2D Brownian motion nearly uncorrelated
dτ, κ = 0, 0.1

### Case 1
Initial position and direction

In [ ]:
x₀ = zeros(2)
θ₀ = 0;

Simulation

In [ ]:
tt, xx = velocityjump_2d(####)

In [ ]:
using Plots
xbound = maximum(abs.(xx[:, 1]))
ybound = maximum(abs.(xx[:, 2]))
plot(xx[:, 2], xx[:, 1],  xlims=(-ybound, ybound), ylims = (-xbound, xbound), label="Randowm walk", xlabel = "y", ylabel = "x", 
    title = "dθₖ ~ vonMises(μ, κ),             μ=-dτ*sin(θₖ₋₁), dτ: ".*"$(dτ)".*" and κ: ".*"$(κ)", size=(800,500))
scatter!([xx[1, 1]], [xx[1, 2]], markersize=7, label="Initial position", legend=:bottomright)

### Case 2
Initial position and direction

In [ ]:
# correlated, but not preferred direction
#dτ, κ = 0, 2

# correlated, with preferred direction
dτ, κ = 0.3, 4

Simulation

In [ ]:
tt, xx = velocityjump_2d(####)

In [ ]:
x₀ = zeros(2)
θ₀ = 0;

In [ ]:
using Plots
xbound = maximum(abs.(xx[:, 1]))
ybound = maximum(abs.(xx[:, 2]))
plot(xx[:, 2], xx[:, 1], label="Randowm walk", xlabel = "y", ylabel = "x", 
    title = "dθₖ ~ vonMises(μ, κ),             μ=-dτ*sin(θₖ₋₁), dτ: ".*"$(dτ)".*" and κ: ".*"$(κ)", size=(800,500))
scatter!([xx[1, 1]], [xx[1, 2]], markersize=7, label="Initial position", legend=:bottomright)